In [ ]:
%matplotlib inline

import sys
sys.path.insert(0, '../python/casadi_f16')
import f16
import control
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg

from analysis import loop_analysis, rlocus, bode

plt.rcParams['figure.figsize'] = (10, 10)

# Pitch-Rate CAS Design

* Example 4.5-1
* pg. 310

In [ ]:
p = f16.Parameters()
x0, u0 = f16.trim(x=f16.State(VT=550), p=p, phi_dot=0, theta_dot=0, psi_dot=np.radians(20), gam=0)

In [ ]:
np.radians(20)

In [ ]:
def f_control(t, x):
    return f16.Control(
        thtl=u0.thtl,
        ail_cmd_deg=u0.ail_cmd_deg,
        elv_cmd_deg=u0.elv_cmd_deg + 0*np.sin(2*np.pi*2*t),
        rdr_cmd_deg=u0.rdr_cmd_deg)

f_control(0, x0)

In [ ]:
res = f16.simulate(x0, f_control, p, 0, 20, 0.01)
plt.plot(res['t'], np.rad2deg(res['x'][:, f16.State().name_to_index('alpha')]))
plt.xlabel('t ,sec')
plt.ylabel(r'$\alpha$, deg')
plt.grid()
plt.title('angle of attack')

In [ ]:
res = f16.simulate(x0, f_control, p, 0, 20, 0.01)

plt.plot(res['x'][:, f16.State().name_to_index('p_E')], res['x'][:, f16.State().name_to_index('p_N')])
plt.axis('equal');
plt.xlabel('East, ft')
plt.ylabel('North, ft')
plt.grid()
plt.title('trajectory')

In [ ]:
def f_control(t, x):
    print(x)
    return f16.Control(
        thtl=u0.thtl,
        ail_cmd_deg=u0.ail_cmd_deg,
        elv_cmd_deg=u0.elv_cmd_deg + 1*np.sin(2*np.pi*2*t),
        rdr_cmd_deg=u0.rdr_cmd_deg)

In [ ]:
def select(n, i):
    D = np.zeros((1, n))
    D[0, i] = 1
    return control.ss([], [], [], D)

In [ ]:
ss = f16.linearize(x0, u0, p)
s = control.tf([1, 0], [0, 1])
G = -(180/np.pi)*ss.sub_system(x=['alpha', 'Q', 'elv_deg'],
                  u=['elv_cmd_deg'], y=['alpha', 'Q']).to_control()
sys3 = control.feedback(G, 0.2*10/(s+10)*select(2, 0))
sys3

In [ ]:
from analysis import rlocus
H = (10/(s+10))*select(2, 0)
plt.figure()
kalpha = 0.2
rlocus('alpha', control.minreal(H*G), kvect=np.linspace(0, 10, 1000), k=kalpha);
plt.plot([0, -2], [0, 2], '--')
plt.axis([-21, 0, -8, 8])

In [ ]:
plt.figure()
sys3 = control.feedback(G, kalpha*(10/(s+10))*select(2, 0))
rlocus('p', (s+3)/s*sys3[1, 0], kvect=np.linspace(0, 1, 1000), k=0.4)
zeta_des = 0.707
plt.plot([0, -10], [0, 10*np.cos(zeta_des)], '--')
#plt.axis([-20, 0, -5, 5]) k changes location of close root pole

In [ ]:
H_PI = 2.28+ 1 *(0.7/s) + 1*0*s
H_PI
# Wroking Values H_PI = 2 + 1 *(1/s) + 1*0*s

In [ ]:
Go = H_PI*sys3[1, 0] #open loop tf
Go

In [ ]:
Gc = control.feedback(Go, 1)
#Gc = Go/(1+Go) closed loop

In [ ]:
t, y = control.step_response(Gc, T=np.linspace(0, 1, 1000))
plt.figure(figsize=(10, 5))
plt.plot(t, 1*y)
plt.grid()
plt.show()
max(y)
#This is so I can see the over shoot. It seems it is the limiting factor suppressing my rise time from being better.

In [ ]:
bode('test', Go, omega=np.logspace(-2, 4), margins=True, Hz=True)

In [ ]:
control.nyquist(Go);

In [ ]:
t, y = control.step_response(Gc, T=np.linspace(0, 1, 1000))
plt.figure(figsize=(10, 5))
plt.plot(t, 10*y)
plt.grid()
plt.show()

In [ ]:
t, y = control.step_response(Gc, T=np.linspace(0, 1, 1000))
plt.figure(figsize=(10, 5))
plt.plot(t, 100*y)
plt.grid()
plt.show()